In [43]:
import cassandra
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement
from cassandra import ConsistencyLevel

In [6]:
#ip wezla
cluster = Cluster(["172.17.0.2"])

In [7]:
#polaczenie
session = cluster.connect()

In [44]:
#ustawienie heyspace'u
session.set_keyspace("ex1")

In [45]:
#deprecated - nie powinno sie uzywac
#consistency level stosujemy per zapytanie do bazy
session.default_consistency_level = ConsistencyLevel.ONE

/home/users/lrydzkow/ENV/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Setting the consistency level at the session level will be removed in 4.0. Consider using execution profiles and setting the desired consitency level to the EXEC_PROFILE_DEFAULT profile.
  """Entry point for launching an IPython kernel.


In [9]:
#zapytanie do bazy
results = session.execute("select * from scores")

In [10]:
results.column_types

[cassandra.cqltypes.VarcharType,
 cassandra.cqltypes.VarcharType,
 cassandra.cqltypes.Int32Type,
 cassandra.cqltypes.Int32Type]

In [11]:
for row in results:
    print(row)

Row(user='pcmanus', game='Coup', year=2015, score=2000)
Row(user='yukim', game='Coup', year=2015, score=2250)
Row(user='jmckenzie', game='Coup', year=2015, score=2000)
Row(user='iamaleksey', game='Coup', year=2015, score=2500)
Row(user='tjake', game='Coup', year=2015, score=1000)
Row(user='jbellis', game='Coup', year=2015, score=1750)


In [16]:
results = session.execute("select * from scores")
for row in results: 
    #print(row)
    print(f"Zawodnik {row.user} uzyskal {row.score} w roku {row.year}")

Zawodnik pcmanus uzyskal 2000 w roku 2015
Zawodnik yukim uzyskal 2250 w roku 2015
Zawodnik jmckenzie uzyskal 2000 w roku 2015
Zawodnik iamaleksey uzyskal 2500 w roku 2015
Zawodnik tjake uzyskal 1000 w roku 2015
Zawodnik jbellis uzyskal 1750 w roku 2015


In [19]:
results = session.execute("select * from scores WHERE user='pcmanus'")
for row in results: 
    #print(row)
    print(f"Zawodnik {row.user} uzyskal {row.score} w roku {row.year}")

Zawodnik pcmanus uzyskal 2000 w roku 2015


In [21]:
player = "pcmanus"
results = session.execute("select * from scores WHERE user=%s", [player])
for row in results: 
    #print(row)
    print(f"Zawodnik {row.user} uzyskal {row.score} w roku {row.year}")

Zawodnik pcmanus uzyskal 2000 w roku 2015


In [23]:
player = "pcmanus"
results = session.execute("select * from scores WHERE user=%s", (player,))
for row in results: 
    #print(row)
    print(f"Zawodnik {row.user} uzyskal {row.score} w roku {row.year}")

Zawodnik pcmanus uzyskal 2000 w roku 2015


In [29]:
arguments = {
    "player": "pcmanus"
}

results = session.execute("select * from scores WHERE user=%(player)s", arguments)
for row in results: 
    #print(row)
    print(f"Zawodnik {row.user} uzyskal {row.score} w roku {row.year}")

Zawodnik pcmanus uzyskal 2000 w roku 2015


In [40]:
player  = "pcmanus"
results = session.execute("select * from scores WHERE user=%(player)s", {"player":player})
for row in results: 
    #print(row)
    print(f"Zawodnik {row.user} uzyskal {row.score} w roku {row.year}")

Zawodnik pcmanus uzyskal 2000 w roku 2015


In [36]:
player  = "pcmanus"
stmt = SimpleStatement("select * from scores WHERE user=%s")
results  = session.execute(stmt, [player])
for row in results:
    print(row)

Row(user='pcmanus', game='Coup', year=2015, score=2000)


In [42]:
pwd

'/home/users/lrydzkow'

In [46]:
#ustawienie consistency level

player  = "pcmanus"
stmt = SimpleStatement("select * from scores WHERE user=%s", consistency_level= ConsistencyLevel.ONE)
results  = session.execute(stmt, [player])
for row in results:
    print(row)

Row(user='pcmanus', game='Coup', year=2015, score=2000)


In [47]:
#przy jednym wezle consisteny level two to nie zadziala
player  = "pcmanus"
stmt = SimpleStatement("select * from scores WHERE user=%s", consistency_level= ConsistencyLevel.TWO)
results  = session.execute(stmt, [player])
for row in results:
    print(row)

NoHostAvailable: ('Unable to complete the operation against any hosts', {<Host: 172.17.0.2:9042 datacenter1>: Unavailable('Error from server: code=1000 [Unavailable exception] message="Cannot achieve consistency level TWO" info={\'consistency\': \'TWO\', \'required_replicas\': 2, \'alive_replicas\': 1}')})

In [75]:
stmt  = SimpleStatement("INSERT INTO scores (user,game,year,score) VALUES (%s, %s, %s, %s)")
results = session.execute(stmt,["pcmanus","Coup2020",2020,3000])


In [77]:
for row in results:
    print(row)

In [78]:
player  = "pcmanus"
stmt = SimpleStatement("select * from scores WHERE user=%s", consistency_level= ConsistencyLevel.ONE)
results  = session.execute(stmt, [player])
for row in results:
    print(row)

Row(user='pcmanus', game='Coup', year=2015, score=2000)
Row(user='pcmanus', game='Coup2020', year=2020, score=3000)


In [80]:
#dodanie nowej kolumny
session.execute("ALTER TABLE scores ADD address map<text, text>")

In [81]:
#update wartosci
session.execute("UPDATE scores set address['home'] = 'Warsaw' WHERE user ='pcmanus' AND game = 'Coup' and year = 2015")

In [87]:
#od pythona 3.6 wszystkie slowniki sa w kolejnosci 
results = session.execute(" select address from scores WHERE user = 'pcmanus'")
for row in results:
    print(row.address)

{'home': 'Warsaw'}
None


In [92]:
#z wykorzystaniem slownika
address = {
    'home': 'Warsaw',
    'work': 'Piaseczno'   
}

session.execute("UPDATE scores set address = %s WHERE user ='pcmanus' AND game = 'Coup' and year = 2015", [address])

results = session.execute(" select address from scores WHERE user = 'pcmanus'")
for row in results:
    print(row.address)

{'home': 'Warsaw', 'work': 'Piaseczno'}
None


In [112]:
#session.exectue("ALTER TABLE scores DROP detailed_scores")
#session.execute("ALTER TABLE scores ADD detailed_scores2 map<text, frozen<set<int>>>")

detailed_scores = {
    'game': {1,2,3,4,4},
    'game2': {40,50,60,60}
}    
session.execute("UPDATE scores set detailed_scores2 = %s WHERE user='pcmanus' AND game='Coup' AND year=2015", [detailed_scores])
    
results = session.execute(" select detailed_scores2 from scores WHERE user = 'pcmanus'")
for row in results:
    print(row.detailed_scores2)


{'game': SortedSet([1, 2, 3, 4]), 'game2': SortedSet([40, 50, 60])}
None
